In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


# Load Datas

In [3]:
df = pd.read_csv('sdn_ml.csv')
df.index = df['timestamp']
df.drop('timestamp', axis=1, inplace=True)
df.head()

,hour,minut,second,day,month,delay,sensor_id,temperature,label
timestamp,,,,,,,,,
2004-03-31 02:58:45,2,58,45,31,3,0.0,3,122.153,0
2004-02-28 00:58:46,0,58,46,28,2,0.0,3,20.204,0
2004-03-30 16:04:46,16,4,46,30,3,0.0,3,122.153,0
2004-03-30 14:43:05,14,43,5,30,3,0.0,3,122.153,0
2004-03-30 16:06:06,16,6,6,30,3,0.0,3,122.153,0


# Separing dataset

In [4]:
train_size = int(len(df) * 0.9)
test_size = len(df) - train_size
train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]


# Normalizing

In [5]:
from sklearn.preprocessing import RobustScaler

In [6]:
f_columns = ['delay','temperature']

f_transformer = RobustScaler()
lb_transformer = RobustScaler()

f_transformer = f_transformer.fit(train[f_columns].to_numpy())
train.loc[:,f_columns] = f_transformer.transform(train[f_columns].to_numpy())

#lb_transformer = lb_transformer.fit(train[['label']])
#train['label'] = lb_transformer.transform(train[['label']])

C:\Users\silva\anaconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [7]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i: (i+time_steps),0:8].to_numpy()
        Xs.append(v)
        ys.append(y.iloc[i+time_steps])
    return np.array(Xs), np.array(ys)

In [8]:
TIME_STEPS = 20

X_train,Y_train = create_dataset(train, train.label, time_steps=TIME_STEPS)
X_test,Y_test = create_dataset(test, test.label, time_steps=TIME_STEPS)

In [9]:
#deoeting label
# X_train.drop('label', axis=1, inplace=True)
# X_test.drop('label', axis=1, inplace=True)


In [10]:
X_train

array([[[ 2.        , 58.        , 45.        , ..., -0.5       ,
          3.        , -0.83541017],
        [ 0.        , 58.        , 46.        , ..., -0.5       ,
          3.        , -1.81944538],
        [16.        ,  4.        , 46.        , ..., -0.5       ,
          3.        , -0.83541017],
        ...,
        [ 1.        ,  8.        , 16.        , ...,  0.        ,
          3.        , -0.14567146],
        [ 1.        ,  8.        , 46.        , ...,  0.        ,
          3.        , -0.14567146],
        [ 1.        ,  9.        , 16.        , ...,  0.        ,
          3.        , -0.14661738]],

       [[ 0.        , 58.        , 46.        , ..., -0.5       ,
          3.        , -1.81944538],
        [16.        ,  4.        , 46.        , ..., -0.5       ,
          3.        , -0.83541017],
        [14.        , 43.        ,  5.        , ..., -0.5       ,
          3.        , -0.83541017],
        ...,
        [ 1.        ,  8.        , 46.        , ...,  

In [11]:
print(type(X_train), type(Y_train))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


# Training

In [12]:
model = keras.Sequential()
model.add(
    keras.layers.Bidirectional(
        keras.layers.LSTM(
            units=128,
            input_shape=(X_train.shape[1],X_train.shape[2])
        )
    ))
model.add(keras.layers.Dense(units=128))
model.add(keras.layers.Dense(units=128))
# model.add(keras.layers.Dense(units=128))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=1))
# model.add(tf.keras.layers.ThresholdedReLU(theta=1))

# Compilation

In [13]:
loss ="mse"
optim = tf.keras.optimizers.Adam(
    learning_rate=0.001)
metrics=["accuracy"]

In [14]:
model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [ ]:
history = model.fit(
    X_train, Y_train, 
    epochs=50, 
    batch_size= 20,
    validation_split=0.1,
    shuffle=False
)

Epoch 1/50
1870/1870 [==============================] - 40s 19ms/step - loss: 0.1620 - accuracy: 0.8064 - val_loss: 0.0918 - val_accuracy: 0.9045
Epoch 2/50
1870/1870 [==============================] - 28s 15ms/step - loss: 0.0829 - accuracy: 0.9101 - val_loss: 0.0793 - val_accuracy: 0.8987
Epoch 3/50
1870/1870 [==============================] - 28s 15ms/step - loss: 0.0765 - accuracy: 0.9154 - val_loss: 0.0791 - val_accuracy: 0.8989
Epoch 4/50
1870/1870 [==============================] - 32s 17ms/step - loss: 0.0741 - accuracy: 0.9170 - val_loss: 0.0800 - val_accuracy: 0.8989
Epoch 5/50
1870/1870 [==============================] - 34s 18ms/step - loss: 0.0729 - accuracy: 0.9172 - val_loss: 0.0806 - val_accuracy: 0.8989
Epoch 6/50
1870/1870 [==============================] - 32s 17ms/step - loss: 0.0720 - accuracy: 0.9176 - val_loss: 0.0816 - val_accuracy: 0.8989
Epoch 7/50
1870/1870 [==============================] - 29s 15ms/step - loss: 0.0713 - accuracy: 0.9182 - val_loss: 0.0812 -

In [ ]:
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend();

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend();

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
plt.plot(Y_test, marker='.', label='true')
plt.plot(y_pred,'r',marker='.', label='predicted')
plt.legend();

In [ ]:
y_pred

In [ ]:
Y_test

In [ ]:
# y_train_inv = lb_transformer.inverse_transform(Y_train.reshape(1,-1)) 
# y_test_inv = lb_transformer.inverse_transform(Y_test.reshape(1,-1))
# y_pred_inv = lb_transformer.inverse_transform(y_pred)